In [24]:
import requests
import wikipediaapi
import json
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from bs4 import BeautifulSoup


llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4",
)

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for."
    )

# DuckDuckGo 검색 툴
class CustomDDGSearchTool(BaseTool):
    name = "DDGSearchTool"
    description = """
    Use this tool to find the documents using DuckDuckGo Search Engine.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query: str):
        try:
            search_url = f"https://html.duckduckgo.com/html/?q={query}"
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(search_url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                results = soup.find_all('a', class_='result__a')
                results = "\n".join([result.get_text() for result in results])
                save_to_txt(f"research_DDG.txt", results)
                return results
            else:
                return f"Error fetching results from DuckDuckGo with status code {response.status_code}."
        except Exception as e:
            return f"Error fetching results from DuckDuckGo: {str(e)}"


class WikiPediaSearchTool(BaseTool):
    name = "WikiPediaSearchTool"
    description = """
    Use this tool to find the documents using Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query: str):
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            wiki_wiki = wikipediaapi.Wikipedia('en', headers=headers)  
            page = wiki_wiki.page(query)
            if page.exists():
                save_to_txt(f"research_WIKI.txt", page.text)
                return page.text
            else:
                return f"No Wikipedia page found for {query}."
        except Exception as e:
            return f"Error fetching results from Wikipedia: {str(e)}"


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CustomDDGSearchTool(),
        WikiPediaSearchTool(),
    ],
)


def save_to_txt(filename, content):
    # content가 dict일 경우 JSON으로
    if isinstance(content, dict):
        content = json.dumps(content, indent=4, ensure_ascii=False)
    with open(filename, 'a', encoding='utf-8') as f:
        f.write(content)
    print(f"Results saved to {filename}")


query = "XZ backdoor"
prompt = f"Research about the {query}."
result = agent.invoke(prompt)

save_to_txt(f"research_{query}.txt", result)



> Entering new AgentExecutor chain...

Invoking: `DDGSearchTool` with `{'query': 'XZ backdoor'}`


Results saved to research_DDG.txt
XZ Utils backdoor - Wikipedia
The XZ Backdoor: Everything You Need to Know - WIRED
The XZ Backdoor explained - Cybernews
The Mystery of 'Jia Tan,' the XZ Backdoor Mastermind - WIRED
All about the xz-utils backdoor | Kali Linux Blog
How one volunteer stopped a backdoor from exposing Linux systems ...
XZ Utils Backdoor — Everything You Need to Know, and What ... - Akamai
Backdoor found in widely used Linux utility targets encrypted SSH ...
The XZ Utils backdoor (CVE-2024-3094): Everything you need to ... - Datadog
XZ Utils Supply Chain Attack: A Threat Actor Spent Two Years to ...
CVE-2024-3094 XZ Backdoor: All you need to know - JFrog
XZ Backdoor: The full story in one place - daily.dev
XZ Utils Backdoor | Threat Actor Planned to Inject ... - SentinelOne
500ms to midnight: XZ / liblzma backdoor - Elastic
The XZ Utils Linux Backdoor: How It Happened & Wha